# PredNet feature extraction

In this notebook we extract representations from PredNet models trained on videos. We also generate RDM submissions for fMRI and MEG data.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Copying code and supporting files from Google Drive...

In [0]:
!unzip -q gdrive/My\ Drive/algonauts/algonauts-master.zip -d /content/
!unzip -q gdrive/My\ Drive/algonauts/algonauts-dataset.zip -d /content/
!mv algonauts-master/* /content/
!cp Feature_Extract/create_RDMs.py /content/
!cp prednet/*.py /content/
!cp -r gdrive/My\ Drive/algonauts/prednet-models /content/

In [0]:
#!cp -r gdrive/My\ Drive/algonauts/prednet-feats /content/
!mkdir -p prednet-feats/92images
!cp -r gdrive/My\ Drive/algonauts/prednet-feats/92images/alexnet /content/prednet-feats/92images
!mkdir -p prednet-feats/78images
!cp -r gdrive/My\ Drive/algonauts/prednet-feats/78images/alexnet /content/prednet-feats/78images

In [0]:
import h5py
import numpy as np
import scipy.io as sio
from skimage.measure import block_reduce
import os
import glob
import zipfile
from data import DataGenerator
np.random.seed(42)

Using TensorFlow backend.


## Extract PredNet features

In [0]:
from keras import backend as K
from keras.layers import Input
from keras.models import Model
import prednet_model
import prednet_rdm
import utils


def extract_features(images_path, hidden_dims=[256], 
                     use_multiprocessing=False, workers=1, 
                     results_dir='./results', input_shape=[10, 256, 256, 3], 
                     mask_value=None, data_format=K.image_data_format(), 
                     **config):
  
    if not os.path.exists(results_dir): 
      os.makedirs(results_dir)
      
    input_width = input_shape[1]
    input_height = input_shape[2]
    prednet = prednet_model.create_model(**config)
    #if weights_file:
    #    prednet_model.load_prednet_weights(weights_file, prednet)
    prednet.summary()
    
    layer_config = prednet.layers[1].get_config()
    data_format = layer_config['data_format'] if 'data_format' in layer_config else layer_config['dim_ordering']
    
    def resize(img):
        return utils.resize_img(img, target_size=(config['input_height'], 
                                                  config['input_width']))

    print('Creating generator...')
    data_generator = DataGenerator(classes=None,
                                   seq_length=config['n_timesteps'],
                                   target_size=None,
                                   rescale=config['rescale'],
                                   fn_preprocess=resize,
                                   batch_size=config['batch_size'], 
                                   shuffle=False,
                                   return_sources=True,
                                   data_format=data_format)
    
    data_generator = data_generator.flow_from_directory(images_path)
    
    if len(data_generator) == 0:
        return
    
    n_batches = len(data_generator)
    print('Number of batches: {}'.format(n_batches))
    
    data_iterator = iter(data_generator)
    for img_i in range(n_batches):
        img_data, y_, sources_ = next(data_iterator)
    
        features = {}
        prednet_rep = prednet.predict(img_data, data_generator.batch_size)
        # prednet_rep = prednet_rep[:,:, 0:prednet.output_shape[-1]]
        print('prednet rep shape:', prednet_rep.shape)
        
        rep_layers = prednet.layers[1].unflatten_features(img_data.shape, prednet_rep)
        rep = []
        
        width_index = 3
        channel_index = -1
        block_size = [1, 1, 1, 1, 1]
        if data_format == 'channels_first':
            width_index = -1
            channel_index = 2
        
        for layer_index, rep_layer in enumerate(rep_layers):
            for timestep_index, timestep_rep in enumerate(rep_layer[0]):
                # getting only the last timestep
                #if timestep_index < config['n_timesteps']-1:
                #    continue
                
                features[f'prednet_{layer_index}_timestep_{timestep_index}'] = timestep_rep
                print(f'prednet_{layer_index}_timestep_{timestep_index} shape: {timestep_rep.shape}')
        
            # Do avg spatial pooling to make all representation layers have
            # the same dimension as the higher-level representation layer.
            ratio = int(rep_layer.shape[width_index] / rep_layers[-1].shape[width_index])
            if ratio > 1:
                block_size[width_index-1] = ratio
                block_size[width_index] = ratio
                rep_layer = block_reduce(rep_layer, block_size=tuple(block_size), func=np.mean)
            rep.append(rep_layer)
            
        rep = np.concatenate(rep, axis=channel_index)
        
        if data_format == 'channels_first':
            rep = np.transpose(rep, (0, 1, 3, 4, 2))

        print('prednet all shape', rep[0][-1].shape)
        
        # features[f'prednet_all'] = rep[0][-1]
                
        feature_path = os.path.join(results_dir, f'image_{img_i+1:03d}.mat')
        print('Saving feature:', feature_path)
        sio.savemat(feature_path, features)


def extract_features_finetuned(images_path, hidden_dims=[32], 
                               use_multiprocessing=False, workers=1, 
                               results_dir='./results', input_shape=[10, 256, 256, 3], 
                               mask_value=None, data_format=K.image_data_format(), 
                               model_json_file=None, model_weights_file=None, model_file=None,
                               **config):
  
    if not os.path.exists(results_dir): 
      os.makedirs(results_dir)
      
    #print('Loading model: {}'.format(model_json_file))
    model = prednet_rdm.create_model(input_shape, hidden_dims, 
                                     mask_value=mask_value, **config)
    # model.load_weights(model_weights_file)
    
    config['input_channels'] = input_shape[3]
    config['input_width'] = input_shape[1]
    config['input_height'] = input_shape[2]
    config['model_json_file'] = None
    config['output_mode'] = 'representation'
    
    if 'prednet_json_file' in config:
        config['model_weights_file'] = config['prednet_weights_file']
        config['model_json_file'] = config['prednet_json_file']
        prednet = prednet_model.create_model(**config)
    else:
        weights_file = config['prednet_weights_file']
        config['model_weights_file'] = None
        prednet = prednet_model.create_model(**config)
    
        # Unfortunately, keras 'model.load_weights' will not always work
        # with prednet layers, so we have to load the weights 'manually' in this case.
        prednet_model.load_prednet_weights(weights_file, prednet)
    
    prednet.summary()
    prednet_layer = prednet.layers[1]
    
    prednet_out = Input(shape=prednet.output_shape[1:])
    last_rep = model.get_layer('crop_last_rep_a')(prednet_out)
    lstm_layer = model.get_layer('lstm')
    lstm_out = lstm_layer(last_rep)
    lstm_model = Model(prednet_out, lstm_out, name='lstm_model')    
    lstm_model.load_weights(model_weights_file, by_name=True)
    #load_model_weights(model_weights_file, lstm_layer)
    lstm_model.summary()
    
    layer_config = prednet_layer.get_config()
    data_format = layer_config['data_format'] if 'data_format' in layer_config else layer_config['dim_ordering']
    
    input_width = input_shape[1]
    input_height = input_shape[2]
    
    def resize(img):
        return utils.resize_img(img, target_size=(input_height, input_width))

    print('Creating generator...')
    data_generator = DataGenerator(classes=None,
                                   seq_length=config['n_timesteps'],
                                   target_size=None,
                                   rescale=config['rescale'],
                                   fn_preprocess=resize,
                                   batch_size=config['batch_size'], 
                                   shuffle=False,
                                   return_sources=True,
                                   data_format=data_format)
    
    data_generator = data_generator.flow_from_directory(images_path)
    
    if len(data_generator) == 0:
        return
    
    n_batches = len(data_generator)
    print('Number of batches: {}'.format(n_batches))
    
    data_iterator = iter(data_generator)
    for img_i in range(n_batches):
        img_data, y_, sources = next(data_iterator)
        print(img_i+1, sources[0][0])
    
        assert img_data.shape == (1, config['n_timesteps'], 256, 256, 3), img_data.shape
        
        features = {}
        
        prednet_rep = prednet.predict(img_data, data_generator.batch_size)
        prednet_rep = prednet_rep[:,:, 0:prednet.output_shape[-1]]
        print('prednet rep shape:', prednet_rep.shape)
        
        lstm_rep = lstm_model.predict(prednet_rep, data_generator.batch_size)
        print('lstm rep shape:', lstm_rep.shape)
        features['lstm'] = lstm_rep
        
        rep_layers = prednet_layer.unflatten_features(img_data.shape, prednet_rep)
        rep = []
        
        width_index = 3
        channel_index = -1
        block_size = [1, 1, 1, 1, 1]
        if data_format == 'channels_first':
            width_index = -1
            channel_index = 2

        for layer_index, rep_layer in enumerate(rep_layers):
            for timestep_index, timestep_rep in enumerate(rep_layer[0]):
                # getting only the last timestep
                #if timestep_index < config['n_timesteps']-1:
                #    continue
                
                features[f'prednet_{layer_index}_timestep_{timestep_index}'] = timestep_rep
                print(f'prednet_{layer_index}_timestep_{timestep_index} shape: {timestep_rep.shape}')
                
                # features[f'prednet_{layer_index}_timestep_{timestep_index}_lstm'] = np.concatenate([timestep_rep.ravel(), lstm_rep.ravel()])
                
            # Do avg spatial pooling to make all representation layers have
            # the same dimension as the higher-level representation layer.
            ratio = int(rep_layer.shape[width_index] / rep_layers[-1].shape[width_index])
            if ratio > 1:
                block_size[width_index-1] = ratio
                block_size[width_index] = ratio
                rep_layer = block_reduce(rep_layer, block_size=tuple(block_size), func=np.mean)
            rep.append(rep_layer)
            
        rep = np.concatenate(rep, axis=channel_index)
        
        if data_format == 'channels_first':
            rep = np.transpose(rep, (0, 1, 3, 4, 2))

        print('prednet all shape', rep[0][-1].shape)
        # features[f'prednet_all'] = rep[0][-1]
        # features[f'prednet_all_lstm'] = np.concatenate([rep[0][-1].ravel(), lstm_rep.ravel()])
                
        feature_path = os.path.join(results_dir, f'image_{img_i+1:03d}.mat')
        print('Saving feature:', feature_path)
        sio.savemat(feature_path, features)

## Configuration for several model variants

In [0]:
config = {
    'n_timesteps': 10,
    'batch_size': 1,
    'stateful': False,
    'train': False,
    'rescale': 1./255,
    'input_height': 128,
    'input_width': 160,
    'input_channels': 3
}
config['output_mode'] = 'representation'

In [0]:
config['results_dir'] = '/content/prednet-feats/92images/prednet_random'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_random'
extract_features('/content/Test_Data/', **config)

In [0]:
config['model_json_file'] = '/content/prednet-models/kitti_keras/prednet_kitti_model.json'
config['model_weights_file'] = '/content/prednet-models/kitti_keras/prednet_kitti_weights.hdf5'
config['results_dir'] = '/content/prednet-feats/92images/kitti_keras'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/prednet-feats/78images/kitti_keras'
extract_features('/content/Test_Data/', **config)

In [0]:
config['model_json_file'] = '/content/prednet-models/prednet_kitti__moments__model__3c/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_kitti__moments__model__3c/weights.hdf5'
config['results_dir'] = '/content/prednet-feats/92images/prednet_kitti__moments__model__3c'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_kitti__moments__model__3c'
extract_features('/content/Test_Data/', **config)

In [0]:
config['model_json_file'] = '/content/prednet-models/prednet_kitti__moments__model__10c/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_kitti__moments__model__10c/weights.hdf5'
config['results_dir'] = '/content/prednet-feats/92images/prednet_kitti__moments__model__10c'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_kitti__moments__model__10c'
extract_features('/content/Test_Data/', **config)

In [0]:
config['model_json_file'] = '/content/prednet-models/prednet_kitti__moments__model__full/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_kitti__moments__model__full/weights.hdf5'
config['results_dir'] = '/content/prednet-feats/92images/prednet_kitti__moments__model__full'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_kitti__moments__model__full'
extract_features('/content/Test_Data/', **config)

In [0]:
config['model_json_file'] = '/content/prednet-models/prednet_finetuned_moments__algonauts/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_finetuned_moments__algonauts/weights.hdf5'
config['results_dir'] = '/content/results/feats/92images/prednet_finetuned_moments__algonauts'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/results/feats/78images/prednet_finetuned_moments__algonauts'
extract_features('/content/Test_Data/', **config)

In [0]:
config['model_json_file'] = '/content/prednet-models/prednet_random__algonauts/model.json'
#config['model_weights_file'] = '/content/prednet-models/prednet_random__algonauts/weights.hdf5'
config['results_dir'] = '/content/results/feats/92images/prednet_random__algonauts'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/results/feats/78images/prednet_random__algonauts'
extract_features('/content/Test_Data/', **config)

In [0]:
config['input_height'] = 256
config['input_width'] = 256
config['stack_sizes'] = (48, 96, 192, 192)
config['results_dir'] = '/content/prednet-feats/92images/prednet_moments__model__4_layers_epoch_1'
config['model_json_file'] = '/content/prednet-models/prednet_moments__model__4_layers_epoch_1/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_moments__model__4_layers_epoch_1/weights.hdf5'
extract_features('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_moments__model__4_layers_epoch_1'
extract_features('/content/Test_Data/', **config)

In [0]:
config['input_height'] = 256
config['input_width'] = 256
config['stack_sizes'] = (48, 96, 192)
config['output_mode'] = 'representation_and_error'
config['results_dir'] = '/content/prednet-feats/92images/prednet_random_rdm_10000'
config['model_json_file'] = '/content/prednet-models/prednet_random_rdm_10000/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_random_rdm_10000/weights.hdf5'
config['prednet_json_file'] = '/content/prednet-models/prednet_random_rdm_10000/prednet_model.json'
config['prednet_weights_file'] = '/content/prednet-models/prednet_random_rdm_10000/prednet_weights.h5'
extract_features_finetuned('/content/Training_Data/92_Image_Set/', hidden_dims=[32], **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_random_rdm_10000'
extract_features_finetuned('/content/Test_Data/', hidden_dims=[32], **config)

In [0]:
config['input_height'] = 256
config['input_width'] = 256
config['stack_sizes'] = (48, 96, 192, 192)
config['output_mode'] = 'representation_and_error'
config['results_dir'] = '/content/prednet-feats/92images/prednet_moments_fmri_it_rdm'
config['model_json_file'] = '/content/prednet-models/prednet_moments_fmri_it_rdm/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_moments_fmri_it_rdm/weights.hdf5'
#config['prednet_json_file'] = '/content/prednet-models/prednet_moments_fmri_it_rdm/prednet_model.json'
config['prednet_weights_file'] = '/content/prednet-models/prednet_moments_fmri_it_rdm/prednet_weights.h5'
extract_features_finetuned('/content/Training_Data/92_Image_Set/', hidden_dims=[128], **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_moments_fmri_it_rdm'
extract_features_finetuned('/content/Test_Data/', hidden_dims=[128], **config)

In [0]:
config['input_height'] = 256
config['input_width'] = 256
config['stack_sizes'] = (48, 96, 192, 192)
config['output_mode'] = 'representation_and_error'
config['results_dir'] = '/content/prednet-feats/92images/prednet_moments_meg_late_rdm_10000'
config['model_json_file'] = '/content/prednet-models/prednet_moments_meg_late_rdm_10000/model.json'
config['model_weights_file'] = '/content/prednet-models/prednet_moments_meg_late_rdm_10000/weights.hdf5'
#config['prednet_json_file'] = '/content/prednet-models/prednet_moments_meg_late_rdm_10000/prednet_model.json'
config['prednet_weights_file'] = '/content/prednet-models/prednet_moments_meg_late_rdm_10000/prednet_weights.h5'
#extract_features_finetuned('/content/Training_Data/92_Image_Set/', **config)
config['results_dir'] = '/content/prednet-feats/78images/prednet_moments_meg_late_rdm_10000'
extract_features_finetuned('/content/Test_Data/', **config)

PredNet trainable layers: None
PredNet trainable units: None
Layer a_0 trainable params: 2640
Layer a_1 trainable params: 83040
Layer a_2 trainable params: 331968
Layer a_3 trainable params: 663744
Layer ahat_0 trainable params: 84
Layer ahat_1 trainable params: 20784
Layer ahat_2 trainable params: 83040
Layer ahat_3 trainable params: 331968
Layer ahat_4 trainable params: 331968
Layer c_0 trainable params: 1542
Layer c_1 trainable params: 103728
Layer c_2 trainable params: 414816
Layer c_3 trainable params: 1327296
Layer c_4 trainable params: 995520
Layer f_0 trainable params: 1542
Layer f_1 trainable params: 103728
Layer f_2 trainable params: 414816
Layer f_3 trainable params: 1327296
Layer f_4 trainable params: 995520
Layer i_0 trainable params: 1542
Layer i_1 trainable params: 103728
Layer i_2 trainable params: 414816
Layer i_3 trainable params: 1327296
Layer i_4 trainable params: 995520
Layer o_0 trainable params: 1542
Layer o_1 trainable params: 103728
Layer o_2 trainable params: 

In [0]:
!cp -r /content/prednet-feats/ /content/gdrive/My\ Drive/prednet-feats-$(date +%Y-%m-%dT%H:%M:%S)

## Generate RDMs

In [0]:
def loadmat(matfile):
    try:
        f = h5py.File(matfile)
    except (IOError, OSError):
        return sio.loadmat(matfile)
    else:
        return {name: np.transpose(f.get(name)) for name in f.keys()}

def loadnpy(npyfile):
    return np.load(npyfile)

def load(data_file):
    root, ext = os.path.splitext(data_file)
    return {'.npy': loadnpy,
            '.mat': loadmat
            }.get(ext, loadnpy)(data_file)

human_fmri = loadmat('./Training_Data/118_Image_Set/target_fmri.mat')
human_fmri['EVC_RDMs'].shape

(15, 118, 118)

In [0]:
import create_RDMs
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def get_features_pca(feat_dir):
    activations = sorted(glob.glob(feat_dir + "/*" + ".mat"))
    
    layer_ids = sorted(sio.loadmat(activations[0]).keys())
    layer_feats_pca = {}
    
    for layer_id in layer_ids:
      
        if layer_id in ['__header__', '__version__', '__globals__']:
            continue
            
        print('Processing features for layer', layer_id)
        layer_feats = []
        
        for img_feats_path in activations:
            img_feats = sio.loadmat(img_feats_path)
            layer_feats.append(img_feats[layer_id].ravel())
            
        layer_feats = np.array(layer_feats)
        
        pca = PCA(n_components=0.99, svd_solver='full')
        scaler = StandardScaler()
        feats_scaled = scaler.fit_transform(layer_feats)
        feats_pca = pca.fit_transform(feats_scaled)
        layer_feats_pca[layer_id] = feats_pca
            
    return layer_feats_pca

def get_layers_ncondns(feat_dir):
    """
    to get number of representative layers in the DNN,
    and number of images(conditions).
    Input:
    feat_dir: Directory containing activations generated using generate_features.py
    Output:
    num_layers: number of layers for which activations were generated
    num_condns: number of stimulus images
    PS: This function is specific for activations generated using generate_features.py
    Write your own function in case you use different script to generate features.
    """
    activations = glob.glob(feat_dir + "/*" + ".mat")
    num_condns = len(activations)
    feat=sio.loadmat(activations[0])
    num_layers = 0
    layer_list = []
    for key in feat:
        if key in ['__header__', '__version__', '__globals__']:
            continue
            
        #if 'prednet_2' not in key:
        #    continue
            
        else:
            num_layers+=1
            layer_list.append(key)
    return num_layers,layer_list,num_condns

def create_rdm(save_dir,feat_dir,dist):
    """
    Main function to create RDM from activations
    Input:
    feat_dir: Directory containing activations generated using generate_features.py
    save_dir : directory to save the computed RDM
    dist : dist used for computing RDM (e.g. 1-Pearson's R)

    Output (in submission format):
    The model RDMs for each layer are saved in
        save_dir/layer_name/submit_fMRI.mat to compare with fMRI RDMs
        save_dir/layer_name/submit_MEG.mat to compare with MEG RDMs
    """

    #get number of layers and number of conditions(images) for RDM
    num_layers,layer_list, num_condns = get_layers_ncondns(feat_dir)
    cwd = os.getcwd()
    
    #feats = sio.loadmat(os.path.join(feat_dir, 'feats_pca.mat'))
    #layer_list = sorted(feats.keys())
    
    #num_condns = 0
    #for layer_id in layer_list:
    #    if isinstance(feats[layer_id], np.ndarray) and len(feats[layer_id]) > 0:
    #        num_condns = len(feats[layer_id])
    #        break
            
    # loops over layers and create RDM for each layer
    for layer in range(num_layers):
        layer_id = layer_list[layer]
        if layer_id in ['__header__', '__version__', '__globals__']:
            continue
            
        if 'timestep_0' in layer_id:
            print('Skipping layer', layer_id)
            continue
      
        os.chdir(cwd)
        #RDM is num_condnsxnum_condns matrix, initialized with zeros
        RDM = np.zeros((num_condns,num_condns))

        #save path for RDMs in  challenge submission format
        print(f'Processing layer {layer_id}...')
        RDM_dir = os.path.join(save_dir,layer_id)
        if not os.path.exists(RDM_dir):
            os.makedirs(RDM_dir)
        RDM_filename_meg = os.path.join(RDM_dir,'submit_meg.mat')
        RDM_filename_fmri = os.path.join(RDM_dir,'submit_fmri.mat')
        RDM_filename_meg_zip = os.path.join(RDM_dir,'submit_meg.zip')
        RDM_filename_fmri_zip = os.path.join(RDM_dir,'submit_fmri.zip')
        #RDM loop
        for i in range(num_condns):
            for j in range(num_condns):
                #get feature for image index i and j
                feature_i = create_RDMs.get_features(feat_dir,layer_id,i)
                feature_j = create_RDMs.get_features(feat_dir,layer_id,j)
                
                #feature_i = feats[layer_id][i].ravel()
                #feature_j = feats[layer_id][j].ravel()
                
                # compute distance 1-Pearson's R
                if dist == 'pearson':
                    RDM[i,j] = 1-np.corrcoef(feature_i,feature_j)[0][1]
                else:
                    print("The", dist, "distance measure not implemented, please request through issues")

        #saving RDMs in challenge submission format
        rdm_fmri={}
        rdm_meg={}
        rdm_fmri['EVC_RDMs'] = RDM
        rdm_fmri['IT_RDMs'] = RDM
        rdm_meg['MEG_RDMs_late'] = RDM
        rdm_meg['MEG_RDMs_early'] = RDM
        sio.savemat(RDM_filename_fmri,rdm_fmri)
        sio.savemat(RDM_filename_meg,rdm_meg)

        #creating zipped file for submission
        zipfmri = zipfile.ZipFile(RDM_filename_fmri_zip, 'w')
        zipmeg = zipfile.ZipFile(RDM_filename_meg_zip, 'w')
        os.chdir(RDM_dir)
        zipfmri.write('submit_fmri.mat')
        zipmeg.write('submit_meg.mat')
        zipfmri.close()
        zipmeg.close()
        
    os.chdir(cwd)
    
def create_rdms(feat_dir, rdm_dir):
    """
    Create RDMs for all features (from all models) in folder feat_dir.
    """
    for feat_path in glob.glob(os.path.join(feat_dir, '*')):
        model_name = os.path.basename(feat_path)
        print(f'Calculating RDMs for model {model_name}')

        rdm_path = os.path.join(rdm_dir, model_name)
        if os.path.exists(rdm_path):
            print(f'Skipping model {model_name} as folder already exists: {rdm_path}')
            continue

        create_rdm(os.path.join(rdm_path, 'pearson'), 
                   feat_path, 'pearson')

In [0]:
!rm -rf prednet-rdms/92images/prednet_moments_fmri_it_rdm/

In [0]:
#feats_pca = get_features_pca('prednet-feats/92images/prednet_moments__model__4_layers_epoch_1/')
#sio.savemat('prednet-feats/92images/prednet_moments__model__4_layers_epoch_1__pca/feats_pca.mat', feats_pca)

feat_dir = '/content/results/feats/92images/'
rdm_dir = '/content/results/rdms/92images/'
create_rdms(feat_dir, rdm_dir)

Calculating RDMs for model prednet_finetuned_moments__algonauts
Skipping model prednet_finetuned_moments__algonauts as folder already exists: /content/results/rdms/92images/prednet_finetuned_moments__algonauts
Calculating RDMs for model prednet_random__algonauts
Skipping layer prednet_0_timestep_0
Processing layer prednet_0_timestep_1...
Processing layer prednet_0_timestep_2...
Processing layer prednet_0_timestep_3...
Processing layer prednet_0_timestep_4...
Processing layer prednet_0_timestep_5...
Processing layer prednet_0_timestep_6...
Processing layer prednet_0_timestep_7...
Processing layer prednet_0_timestep_8...
Processing layer prednet_0_timestep_9...
Skipping layer prednet_1_timestep_0
Processing layer prednet_1_timestep_1...
Processing layer prednet_1_timestep_2...
Processing layer prednet_1_timestep_3...
Processing layer prednet_1_timestep_4...
Processing layer prednet_1_timestep_5...
Processing layer prednet_1_timestep_6...
Processing layer prednet_1_timestep_7...
Processin

In [0]:
feat_dir = '/content/results/feats/78images/'
rdm_dir = '/content/results/rdms/78images/'
create_rdms(feat_dir, rdm_dir)

Calculating RDMs for model prednet_finetuned_moments__algonauts
Skipping layer prednet_0_timestep_0
Processing layer prednet_0_timestep_1...
Processing layer prednet_0_timestep_2...
Processing layer prednet_0_timestep_3...
Processing layer prednet_0_timestep_4...
Processing layer prednet_0_timestep_5...
Processing layer prednet_0_timestep_6...
Processing layer prednet_0_timestep_7...
Processing layer prednet_0_timestep_8...
Processing layer prednet_0_timestep_9...
Skipping layer prednet_1_timestep_0
Processing layer prednet_1_timestep_1...
Processing layer prednet_1_timestep_2...
Processing layer prednet_1_timestep_3...
Processing layer prednet_1_timestep_4...
Processing layer prednet_1_timestep_5...
Processing layer prednet_1_timestep_6...
Processing layer prednet_1_timestep_7...
Processing layer prednet_1_timestep_8...
Processing layer prednet_1_timestep_9...
Skipping layer prednet_2_timestep_0
Processing layer prednet_2_timestep_1...
Processing layer prednet_2_timestep_2...
Processi

In [0]:
!cp -r /content/results/rdms /content/gdrive/My\ Drive/prednet-rdms-$(date +%Y-%m-%dT%H:%M:%S)

In [0]:
import create_RDMs

def merge_feats(save_dir, feat_dir1, feat_dir2, layers1, layers2, dist):
    
    #get number of layers and number of conditions(images) for RDM
    num_layers, layer_list, num_condns = create_RDMs.get_layers_ncondns(feat_dir1)
    
    if not os.path.exists(save_dir): 
        os.makedirs(save_dir)
    
    for img_i in range(num_condns):
        features = {}
        
        for layer1 in layers1:
            for layer2 in layers2:
                feature1 = create_RDMs.get_features(feat_dir1, layer1, img_i)
                feature2 = create_RDMs.get_features(feat_dir2, layer2, img_i)
                features[f'{layer1}__{layer2}'] = np.concatenate([feature1, feature2])
        
        feature_path = os.path.join(save_dir, f'image_{img_i+1:03d}.mat')
        # print('Saving feature:', feature_path)
        sio.savemat(feature_path, features)

In [0]:
feat_dir1 = '/content/prednet-feats/78images/prednet_moments_fmri_it_rdm/'
feat_dir2 = '/content/prednet-feats/78images/alexnet/'
save_dir = '/content/results/feats/78images/prednet_moments_fmri_it_rdm__alexnet'
layers1 = ['prednet_4_timestep_9']
layers2 = ['conv2', 'conv4', 'fc8']
merge_feats(save_dir, feat_dir1, feat_dir2, layers1, layers2, 'pearson')

In [0]:
feat_dir1 = '/content/prednet-feats/92images/prednet_kitti__moments__model__10c/'
feat_dir2 = '/content/prednet-feats/92images/alexnet/'
save_dir = '/content/results/feats/92images/prednet_kitti__moments__model__10c__alexnet'
layers1 = ['prednet_3_timestep_9']
layers2 = ['conv2', 'conv4', 'fc8']
merge_feats(save_dir, feat_dir1, feat_dir2, layers1, layers2, 'pearson')

In [0]:
feat_dir1 = '/content/prednet-feats/78images/prednet_moments__model__4_layers_epoch_1/'
feat_dir2 = '/content/prednet-feats/78images/alexnet/'
save_dir = '/content/prednet-feats/78images/prednet_moments__model__4_layers_epoch_1__alexnet'
layers1 = ['prednet_2_timestep_8', 'prednet_2_timestep_6']
layers2 = ['conv4']
merge_feats(save_dir, feat_dir1, feat_dir2, layers1, layers2, 'pearson')